In [1]:
import numpy as np
import pandas as pd 

In [2]:
df=pd.read_csv(r"C:\Users\DELL\Desktop\athlete_events.csv")
region_df=pd.read_csv(r"C:\Users\DELL\Desktop\noc_regions.csv")

In [3]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [86]:
region_df.head()

# Filtering the data for summer season 

In [46]:
df=df[df['Season']=='Summer']

In [47]:
df.head()
# df3.drop(columns=['region_x','region_y','notes_y','notes_x'],inplace=True)

# Merging the data df and region_df

In [48]:
df=df.merge(region_df,on="NOC",how='left')

In [49]:
df.head()

# Number of country paricipated 

In [50]:
df['region'].unique()

# Counting Missing and duplicates values

In [51]:
df.isnull().sum()

In [52]:
df.duplicated().sum()

In [53]:
df.drop_duplicates(inplace=True)

In [54]:
df.duplicated().sum()

# Medal Count on the basis of region

In [55]:
df['Medal'].value_counts()

In [56]:
# creating a dummy dataframe 
dummy=pd.get_dummies(df['Medal'])
dummy

In [57]:
#concatanating dummy dataframe and orginal dataframe
df=pd.concat([df,dummy],axis=1)
df.head()

In [58]:
df.groupby('NOC').sum()[["Gold","Silver","Bronze"]]

In [59]:
df.head()

In [60]:
df[(df['NOC']=="IND") & (df['Medal']=='Gold')].head()

In [61]:
medal_tally=df.drop_duplicates(subset=['Team','NOC','Games','Year','City','Sport','Event','Medal'])

In [62]:
medal_tally=medal_tally.groupby('NOC').sum()[["Gold","Silver","Bronze"]].sort_values('Gold',ascending=False).reset_index()

In [63]:
medal_tally[medal_tally['NOC']=='IND']

In [64]:
medal_tally['Total']=medal_tally['Gold']+medal_tally['Silver']+medal_tally['Bronze']

In [65]:
medal_tally[medal_tally['NOC']=='IND']

# Years when olympics were played

In [66]:
year=df['Year'].unique().tolist()
year.sort()
year

# Country where olympics were played

In [67]:
country=df['region'].unique().tolist()

In [68]:
len(country)

In [69]:
c1=np.unique(df['region'].dropna().values).tolist()

In [70]:
c1.sort()

In [71]:
# c1

In [72]:
medal_df=df.drop_duplicates(subset=['Team','NOC','Games','Year','City','Sport','Event','Medal'])

In [73]:
def fetch_model(year,country):
    flag=0
    if year == 'overall' and country == 'overall':
        temp_df=medal_df
    if year == 'overall' and country != 'overall':
        temp_df=medal_df[medal_df['region']==country]
        flag=1
    if year != 'overall' and country == 'overall':
        temp_df=medal_df[medal_df['Year']==int(year)]
    if year != 'overall' and country != 'overall':
        temp_df=medal_df[(medal_df['Year']==int(year)) & (medal_df['region']==country)]
    if flag==1:    
        X=temp_df.groupby('Year').sum()[['Gold','Silver','Bronze']].sort_values('Year').reset_index()
    else:
        X=temp_df.groupby('region').sum()[['Gold','Silver','Bronze']].sort_values('Gold').reset_index()
        
    X['total']=X['Gold']+X['Silver']+X['Bronze']
    print(X)
         

In [74]:
year=input("Enter Year")
country=input("Enter Country")
fetch_model(year,country)

Enter Year2016
Enter CountryIndia


# No of Sports / Event / City / Years / Athletes Name / Region 

In [75]:
df['Sport'].unique()

In [76]:
df['Sport'].unique().shape[0]

In [77]:
df['Event'].unique().shape

In [78]:
df['City'].unique().shape[0]

In [79]:
df["Year"].unique().shape[0]-1        # 1906 olympic was invalid 

In [80]:
df['Name'].unique().shape

In [81]:
df['region'].unique().shape

In [82]:
No. of participating country over time 

In [89]:
import matplotlib.pyplot as plt

In [90]:
nation_over_time=df.drop_duplicates(["Year","region"])['Year'].value_counts().reset_index().sort_values('Year')

In [91]:
nation_over_time.rename(columns={'count':'No. of Country'},inplace=True)

In [92]:
plt.plot(nation_over_time['Year'],nation_over_time['No. of Country'])
plt.xlabel("Years")
plt.ylabel("No. of Country")
plt.show()

# No. of events over time 

In [93]:
events_count=df.drop_duplicates(["Year","Event"])['Year'].value_counts().reset_index().sort_values('Year')
events_count.rename(columns={'count':'No. of Event'},inplace=True)

In [94]:
plt.plot(events_count['Year'],events_count['No. of Event'],color='g')
plt.xlabel("Years")
plt.ylabel("No. of Events")
plt.show()

# No. of athletes over time

In [95]:
athletes_over_time=df.drop_duplicates(["Year","Name"])['Year'].value_counts().reset_index().sort_values('Year')
athletes_over_time.rename(columns={"count":"Total Athletes"},inplace=True)

In [96]:
plt.plot(athletes_over_time['Year'],athletes_over_time['Total Athletes'],color='g')
plt.xlabel("Years")
plt.ylabel("Total Athletes")
plt.show()

# No. of Sports  over year

In [97]:
X=df.drop_duplicates(["Year","Sport","Event"])

In [101]:
import seaborn as sns
plt.figure(figsize=(20,20))
sns.heatmap(X.pivot_table(index='Sport',columns="Year",values='Event',aggfunc="count").fillna(0).astype(int),annot=True)

# List of Successful athletes 

In [102]:
def successful(df,sport):
    temp_df2=df.dropna(subset=['Medal'])
    if sport!="overall":
        temp_df2 = temp_df2[temp_df2["Sport"]==sport]     
    X=temp_df2['Name'].value_counts().reset_index()
    return X.rename(columns={'count':'Total Medals'}).head(20)

In [103]:
successful(df,'overall')

# Country wise Medal tally per year

In [104]:
temp_df=df.dropna(subset=["Medal"])
temp_df.drop_duplicates(subset=['Team','NOC','Games','Year','City','Sport','Event','Medal'],inplace=True)

In [105]:
new_df=temp_df[temp_df['region']=='India']
final_df=new_df.groupby("Year").count()['Medal'].reset_index()

In [106]:
final_df

In [107]:
plt.plot(final_df['Year'],final_df['Medal'],color='g')
plt.xlabel("Years")
plt.ylabel("Total Medals")
plt.show()

# which country is good at heatmap

In [108]:
new_df=temp_df[temp_df['region']=='UK']
plt.figure(figsize=(15,15))
sns.heatmap(new_df.pivot_table(index='Sport',columns='Year',values='Medal',aggfunc='count').fillna(0),annot=True)

# Analysis of Height vs Weight in different- different sports 

In [109]:
athlete_df=df.drop_duplicates(subset=['Name','region'])

In [110]:
athlete_df['Medal'].fillna("No Medal",inplace=True)

In [111]:
temp_df=athlete_df[athlete_df['Sport']=="Athletics"]

In [112]:
plt.figure(figsize=(15,10))
sns.scatterplot(x=temp_df['Weight'],y=temp_df['Height'],hue=temp_df['Medal'],style=temp_df['Sex'],s=80)

# Participation of Men Vs Women 

In [113]:
Men=athlete_df[athlete_df['Sex']=="M"].groupby("Year").count()['Name'].reset_index()
Women=athlete_df[athlete_df['Sex']=="F"].groupby("Year").count()['Name'].reset_index()

In [114]:
final=Men.merge(Women,on='Year',how='left',inplace=True)

In [115]:
final.rename(columns={'Name_x':'Men','Name_y':'Women'},inplace=True)
final.head()

In [116]:
final.fillna(0,inplace=True)

In [117]:
final.astype(int).head()

In [118]:
plt.plot(final['Year'],final['Men'],label="Men")
plt.plot(final['Year'],final['Women'],label='Women')
plt.xlabel("Year")
plt.ylabel("Total Players")
plt.legend()